In [1]:
import pandas as pd
from CSVtoPD import NANDCommandLoader


In [2]:
TestData = NANDCommandLoader(csv_path="./WL_01.csv")

In [3]:
TestData.load_csv()
TestData.show_head()

[INFO] CSV 파일이 성공적으로 로드되었습니다: ./WL_01.csv
   index  time      CMD     OPT_1    OPT_2    OPT_3 OPT_4 OPT_5 OPT_6 OPT_7  OPT_8  TOP_9  CH  BANK  DESCRIPTION
0      1   100  CMD(70)  DATA(80)      NaN      NaN   NaN   NaN   NaN   NaN    NaN    NaN   0     0  Read Status
1      2   200  CMD(00)   ADD(00)  ADD(01)  ADD(02)   NaN   NaN   NaN   NaN    NaN    NaN   0     0        ERASE
2      3   300  CMD(70)  DATA(80)      NaN      NaN   NaN   NaN   NaN   NaN    NaN    NaN   0     0  Read Status
3      4   400  CMD(DA)       NaN      NaN      NaN   NaN   NaN   NaN   NaN    NaN    NaN   0     0   SLC Enable
4      5   600  CMD(70)  DATA(81)      NaN      NaN   NaN   NaN   NaN   NaN    NaN    NaN   0     0  Read Status


In [4]:
import json
import math
json_data = TestData.df.to_dict(orient='records')
#print(json.dumps(json_data, indent=4 ))
# nan 값을 None으로 변환하여 json 변환 시 필드가 없는 것으로 처리

def remove_nan(d):
    return {k: v for k, v in d.items() if not (isinstance(v, float) and math.isnan(v))}

json_data_no_nan = [remove_nan(item) for item in json_data]

# JSON 파일로 저장
with open('./WL_01.json', 'w', encoding='utf-8') as f:
    json.dump(json_data_no_nan, f, indent=4, ensure_ascii=False)

print(f"[INFO] JSON 파일로 저장되었습니다: {'./WL_01.json'}")

[INFO] JSON 파일로 저장되었습니다: ./WL_01.json


In [5]:
import sys
from PyQt5.QtWidgets import QApplication, QGraphicsScene, QGraphicsView, QGraphicsRectItem, QGraphicsTextItem
from PyQt5.QtGui import QColor, QBrush
from PyQt5.QtCore import QRectF
from PyQt5.QtGui import QPolygonF, QPen
from PyQt5.QtCore import QPointF
# PyQt5를 사용하여 플랫 NAND 시퀀스 다이어그램 그리기

def draw_flat_sequence_diagram(data):
    app = QApplication(sys.argv)
    scene = QGraphicsScene()
    width = 1200
    height = 200
    x_start = 40
    y = 80
    box_width = 120
    box_height = 60
    gap = 20

    # 시퀀스 리스트 만들기 (CMD/옵션/설명 정보 포함)
    sequence = []
    for item in data:
        # CMD
        sequence.append({
            'text': item['CMD'],
            'type': 'CMD',
            'desc': item.get('DESCRIPTION', ''),
            'index': item.get('index', '')
        })
        # OPT_1 ~ OPT_8, TOP_9
        for k in ['OPT_1','OPT_2','OPT_3','OPT_4','OPT_5','OPT_6','OPT_7','OPT_8','TOP_9']:
            if k in item and item[k] is not None and str(item[k]) != 'nan':
                val = item[k]
                if str(val).startswith('ADD'):
                    typ = 'ADD'
                elif str(val).startswith('DATA'):
                    typ = 'DATA'
                else:
                    typ = 'ETC'
                sequence.append({
                    'text': val,
                    'type': typ,
                    'desc': '',
                    'index': ''
                })

    # 색상 매핑
    color_map = {
        'CMD': QColor(220, 250, 220),
        'ADD': QColor(220, 220, 250),
        'DATA': QColor(250, 220, 220),
        'ETC': QColor(240, 240, 240)
    }

    for i, entry in enumerate(sequence):
        x = x_start + i * (box_width + gap)
        rect = QGraphicsRectItem(QRectF(x, y, box_width, box_height))
        rect.setBrush(QBrush(color_map.get(entry['type'], QColor(240, 240, 240))))
        scene.addItem(rect)

        # CMD 위에 Description과 index 표기
        if entry['type'] == 'CMD':
            desc_text = f"{entry['index']}: {entry['desc']}"
            desc_item = QGraphicsTextItem(desc_text)
            desc_item.setPos(x + 5, y - 30)
            scene.addItem(desc_item)

        # 육각형 그리기 (네모 대신 육각형을 사용)
        # 육각형 좌표 계산
        cx = x + box_width / 2
        cy = y + box_height / 2
        r = min(box_width, box_height) / 2 - 5
        hex_points = [
            QPointF(cx + r * math.cos(math.radians(angle)), cy + r * math.sin(math.radians(angle)))
            for angle in range(0, 360, 60)
        ]
        hexagon = QPolygonF(hex_points)
        hex_item = scene.addPolygon(hexagon, QPen(), QBrush(color_map.get(entry['type'], QColor(240, 240, 240))))

        # 텍스트 추가 (육각형 중앙에 위치)
        txt_item = QGraphicsTextItem(str(entry['text']))
        txt_item.setDefaultTextColor(QColor(0, 0, 0))
        # 텍스트 중앙 정렬을 위해 위치 조정
        text_rect = txt_item.boundingRect()
        txt_item.setPos(cx - text_rect.width() / 2, cy - text_rect.height() / 2)
        scene.addItem(txt_item)

        # 육각형 좌표 계산
        cx = x + box_width / 2
        cy = y + box_height / 2
        r = min(box_width, box_height) / 2 - 5
        hex_points = [
            QPointF(cx + r * math.cos(math.radians(angle)), cy + r * math.sin(math.radians(angle)))
            for angle in range(0, 360, 60)
        ]
        hexagon = QPolygonF(hex_points)
        hex_item = scene.addPolygon(hexagon, QPen(), QBrush(color_map.get(entry['type'], QColor(240, 240, 240))))
        
        # 텍스트 추가 (검정색)
        txt_item = QGraphicsTextItem(str(entry['text']))
        txt_item.setDefaultTextColor(QColor(0, 0, 0))
        txt_item.setPos(x + 10, y + 15)
        scene.addItem(txt_item)

    view = QGraphicsView(scene)
    view.setWindowTitle("Flat NAND Command Sequence Diagram")
    view.resize(width, height)
    view.show()
    sys.exit(app.exec_())

draw_flat_sequence_diagram(json_data_no_nan)


SystemExit: 0

/Users/gdh/.pyenv/versions/3.10.13/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
data = json.load(open('./WL_01.json', 'r', encoding='utf-8'))

with open('./WL_01.txt', 'w', encoding='utf-8') as f:
    for item in data:
        f.write(f"*-{item['index']} : {item['DESCRIPTION']} ----------* \n")
        # OPT_1 ~ OPT_9 값이 nan이더라도 NOP 라인 출력
        opt_keys = ['OPT_1', 'OPT_2', 'OPT_3', 'OPT_4', 'OPT_5', 'OPT_6', 'OPT_7', 'OPT_8', 'TOP_9']
        opts = [str(item.get(k)) for k in opt_keys if item.get(k) is not None]
        f.write(f"NOP {item['CMD']} {' '.join(opts)} \n")
